 # Part 1:  Video Upload and Frame Extraction

1. Install the Required Libraries

In [4]:
import cv2
import os

2. Directory for video uplaod and frame extraction

In [5]:

# Function to create directory for storing frames
def create_frame_dir(video_file):
    frame_dir = os.path.splitext(video_file)[0] + "_frames"
    if not os.path.exists(frame_dir):
        os.makedirs(frame_dir)
    return frame_dir

# Function to extract and save frames from the video
def extract_frames(video_file):
    # Open the video file
    cap = cv2.VideoCapture(video_file)
    
    # Create directory to store the frames
    frame_dir = create_frame_dir(video_file)
    
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # If no frame is captured, exit
        
        # Save frame as image
        frame_path = os.path.join(frame_dir, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)
        
        frame_count += 1
        print(f"Extracted frame {frame_count}")
    
    cap.release()
    print(f"Frame extraction completed. Frames are saved in {frame_dir}")

# Call the function with your video file path
video_file = "Video2.mp4"  # Replace with your video file
extract_frames(video_file)


Extracted frame 1
Extracted frame 2
Extracted frame 3
Extracted frame 4
Extracted frame 5
Extracted frame 6
Extracted frame 7
Extracted frame 8
Extracted frame 9
Extracted frame 10
Extracted frame 11
Extracted frame 12
Extracted frame 13
Extracted frame 14
Extracted frame 15
Extracted frame 16
Extracted frame 17
Extracted frame 18
Extracted frame 19
Extracted frame 20
Extracted frame 21
Extracted frame 22
Extracted frame 23
Extracted frame 24
Extracted frame 25
Extracted frame 26
Extracted frame 27
Extracted frame 28
Extracted frame 29
Extracted frame 30
Extracted frame 31
Extracted frame 32
Extracted frame 33
Extracted frame 34
Extracted frame 35
Extracted frame 36
Extracted frame 37
Extracted frame 38
Extracted frame 39
Extracted frame 40
Extracted frame 41
Extracted frame 42
Extracted frame 43
Extracted frame 44
Extracted frame 45
Extracted frame 46
Extracted frame 47
Extracted frame 48
Extracted frame 49
Extracted frame 50
Extracted frame 51
Extracted frame 52
Extracted frame 53
Ex

# Part 2: Player Detection Using YOLOv5

Step 1: Install YOLOv5

In [6]:
import torch
from pathlib import Path

Step 2: Use YOLOv5 to detect players in frames.

In [8]:

# Load YOLOv5 pre-trained model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Detect players in the extracted frames
def detect_players_in_frames(frame_dir):
    frame_paths = list(Path(frame_dir).glob('*.jpg'))
    
    for frame_path in frame_paths:
        # Perform detection
        results = model(frame_path)
        
        # Display results (bounding boxes around detected players)
        # results.show()  # This will show the image with detection

        # If you want to save the results, uncomment the following line
        # results.save()  # This will save images with bounding boxes to the current directory
        
        # print(f"Detected players in {frame_path}")

# Call the function with the frame directory
frame_dir = os.path.splitext(video_file)[0] + "_frames"  # Replace with your frame directory
detect_players_in_frames(frame_dir)


Using cache found in C:\Users\goel6/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-9 Python-3.12.3 torch-2.4.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\goel6/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\goel6/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\goel6/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\goel6/.cache\torch\hub\ultralytics_yolov5_ma

# Part 3: Tracking the Player Across Frames

Step 1: Install DeepSORT

In [9]:
import numpy as np
import sklearn

print("NumPy version:", np.__version__)
print("scikit-learn version:", sklearn.__version__)



NumPy version: 1.26.4
scikit-learn version: 1.5.0


In [10]:
import sys

In [11]:
from deep_sort_realtime.deepsort_tracker import DeepSort


Step 2: Player tracking with DeepSORT

In [16]:
# Import necessary libraries
import cv2
import torch
from pathlib import Path
import numpy as np

# Initialize the YOLOv5 model (Ensure that it's already set up)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Assuming the DeepSORT object is replaced with a custom tracker or placeholder
class SimpleTracker:
    def __init__(self):
        self.next_id = 0
        self.objects = {}

    def update(self, bboxes):
        new_objects = {}
        for bbox in bboxes:
            new_objects[self.next_id] = bbox
            self.next_id += 1
        self.objects = new_objects
        return [[*bbox, obj_id] for obj_id, bbox in self.objects.items()]

# Initialize a simple tracker
tracker = SimpleTracker()

# Function to track players across frames
def track_players_in_video(video_file):
    cap = cv2.VideoCapture(video_file)
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Exit if no frame is captured

        # Perform detection (similar to part 2)
        results = model(frame)
        bboxes = results.xyxy[0][:, :4].cpu().numpy()  # Bounding boxes
        confs = results.xyxy[0][:, 4].cpu().numpy()    # Confidence scores

        # Perform tracking using a simple tracker (replace deepsort)
        outputs = tracker.update(bboxes)

        # Draw the bounding boxes and track IDs on the frame
        for output in outputs:
            x1, y1, x2, y2, track_id = output[:5]
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv2.putText(frame, f"ID: {track_id}", (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)

        cv2.imshow("Tracking", frame)
        frame_count += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Call the tracking function
video_file = "Video2.mp4"  # Replace with your video file
track_players_in_video(video_file)


Using cache found in C:\Users\goel6/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-9 Python-3.12.3 torch-2.4.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\goel6/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\goel6/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\goel6/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\goel6/.cache\torch\hub\ultralytics_yolov5_ma

# Part 4: Pose Estimation Using OpenPose

Step 1: Install OpenPose

In [17]:
import cv2
import mediapipe as mp

Step 2: Implement Pose Estimation on Extracted Frames

In [20]:
import cv2
import torch
import mediapipe as mp

# Initialize MediaPipe Pose module
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# Load YOLOv5 pre-trained model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Function to perform pose estimation on a single frame
def pose_estimation_on_frame(frame):
    # Convert frame to RGB as MediaPipe works with RGB images
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform pose estimation
    results = pose.process(rgb_frame)

    # Draw keypoints on the frame
    annotated_frame = frame.copy()
    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(annotated_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    return annotated_frame, results.pose_landmarks

# Function to detect players and perform pose estimation
def process_video(video_file):
    cap = cv2.VideoCapture(video_file)

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Exit if no frame is captured

        # Perform detection using YOLOv5
        results = model(frame)
        bboxes = results.xyxy[0].cpu().numpy()  # Bounding boxes

        # Perform pose estimation
        annotated_frame, landmarks = pose_estimation_on_frame(frame)

        # Draw bounding boxes for detected players
        for bbox in bboxes:
            # Check the number of values in bbox
            if len(bbox) == 4:
                x1, y1, x2, y2 = bbox  # If only bounding box coordinates are returned
                conf = 1.0  # Set default confidence if not available
            elif len(bbox) >= 5:
                x1, y1, x2, y2, conf = bbox[:5]  # Get the first five values
            else:
                continue  # Skip if bbox has fewer than 4 values

            # Draw the bounding box
            cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

        # Display the annotated frame with pose landmarks and bounding boxes
        cv2.imshow("Pose Estimation and Player Detection", annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Call the function with your video file
video_file = "Video2.mp4"  # Use the video file specified earlier
process_video(video_file)


Using cache found in C:\Users\goel6/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-9 Python-3.12.3 torch-2.4.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\goel6/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\goel6/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\goel6/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\goel6/.cache\torch\hub\ultralytics_yolov5_ma

# Part 5: Action Recognition

Step 1: Define Actions and Recognition Logic

In [24]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose module
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Function to perform pose estimation on a single frame
def pose_estimation_on_frame(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    annotated_frame = frame.copy()

    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(annotated_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    return annotated_frame, results.pose_landmarks

# Action recognition function
def recognize_action(landmarks):
    if landmarks is not None:
        left_shoulder = landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        left_elbow = landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
        left_wrist = landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]

        # Calculate shoulder angle
        shoulder_angle = calculate_angle(
            (left_shoulder.x, left_shoulder.y),
            (left_elbow.x, left_elbow.y),
            (left_wrist.x, left_wrist.y)
        )

        # Debug outputs for angles
        print(f"Shoulder Angle: {shoulder_angle}")

        # Define action recognition
        if shoulder_angle < 30:  # Shooting
            return "Action: Shooting"
        elif shoulder_angle > 150:  # Running
            return "Action: Running"
        elif 30 <= shoulder_angle <= 150:  # Neutral position
            return "Action: Neutral"  # Example for neutral position
        else:
            return "Action: Unknown"  # Default case

    return "Action: Unknown"

# Process all frames for pose estimation and action recognition
def process_pose_estimation_on_video_with_action_recognition(video_file):
    cap = cv2.VideoCapture(video_file)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        annotated_frame, landmarks = pose_estimation_on_frame(frame)

        # Debug output for landmarks
        if landmarks:
            for idx, landmark in enumerate(landmarks.landmark):
                print(f"Landmark {idx}: ({landmark.x}, {landmark.y})")

        action = recognize_action(landmarks)
        print(action)

        cv2.imshow("Pose Estimation", annotated_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Call the action recognition function with your video
video_file = "Video2.mp4"  # Use the video file specified earlier
process_pose_estimation_on_video_with_action_recognition(video_file)


Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: Unknown
Action: 

# Part 6: Generating Feedback for Players

Step 1: Generate Feedback Based on Recognized Actions

In [25]:
# Part 6: Generating Feedback for Players

def generate_feedback(action):
    feedback = ""
    
    if action == "Action: Shooting":
        feedback = "Feedback: Focus on improving your shooting technique. Aim for accuracy."
    elif action == "Action: Running":
        feedback = "Feedback: Maintain a steady pace and good form while running."
    else:
        feedback = "Feedback: Keep practicing your actions for better performance."
    
    return feedback

# Modify the video processing function to include feedback generation
def process_pose_estimation_on_video_with_feedback(video_file):
    cap = cv2.VideoCapture(video_file)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Perform pose estimation
        annotated_frame, landmarks = pose_estimation_on_frame(frame)

        # Recognize action
        action = recognize_action(landmarks)
        print(action)

        # Generate feedback
        feedback = generate_feedback(action)
        print(feedback)

        # Display the result
        cv2.imshow("Pose Estimation", annotated_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Call the feedback generation function with your video
process_pose_estimation_on_video_with_feedback(video_file)


Action: Unknown
Feedback: Keep practicing your actions for better performance.
Action: Unknown
Feedback: Keep practicing your actions for better performance.
Action: Unknown
Feedback: Keep practicing your actions for better performance.
Action: Unknown
Feedback: Keep practicing your actions for better performance.
Action: Unknown
Feedback: Keep practicing your actions for better performance.
Action: Unknown
Feedback: Keep practicing your actions for better performance.
Action: Unknown
Feedback: Keep practicing your actions for better performance.
Action: Unknown
Feedback: Keep practicing your actions for better performance.
Action: Unknown
Feedback: Keep practicing your actions for better performance.
Action: Unknown
Feedback: Keep practicing your actions for better performance.
Action: Unknown
Feedback: Keep practicing your actions for better performance.
Action: Unknown
Feedback: Keep practicing your actions for better performance.
Action: Unknown
Feedback: Keep practicing your actio

# Now Simple Implementation of getting video and anlysis them

1. Testing and Validation

In [26]:
def analyze_video(video_path):
    # Placeholder for video analysis logic
    # Replace this with actual video analysis code
    # For now, let's just return a dummy feedback
    return f"Analysis completed for {video_path}"

def test_videos(video_list):
    results = []
    for video_path in video_list:
        feedback = analyze_video(video_path)  # Call the video analysis function
        results.append((video_path, feedback))
    return results

# Sample usage with your specified video file
video_list = ['Video2.mp4']  # Use your specific video file
results = test_videos(video_list)
for video, feedback in results:
    print(f"Feedback for {video}: {feedback}")


Feedback for Video2.mp4: Analysis completed for Video2.mp4


In [27]:
action_threshold = 0.5  # Initial threshold

def set_action_threshold(new_threshold):
    global action_threshold
    action_threshold = new_threshold

# Example of updating the threshold
set_action_threshold(0.6)


2. Data Logging

In [28]:
import csv

def log_feedback(video_path, actions, feedback):
    with open('feedback_log.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([video_path, actions, feedback])

# Sample logging
log_feedback('video1.mp4', 'Running, Passing', 'Improve passing accuracy')


In [29]:
import sqlite3

def initialize_db():
    conn = sqlite3.connect('player_analysis.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS performance (
            id INTEGER PRIMARY KEY,
            video_path TEXT,
            actions TEXT,
            feedback TEXT
        )
    ''')
    conn.commit()
    conn.close()

def insert_performance(video_path, actions, feedback):
    conn = sqlite3.connect('player_analysis.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO performance (video_path, actions, feedback) 
        VALUES (?, ?, ?)
    ''', (video_path, actions, feedback))
    conn.commit()
    conn.close()

# Initialize the database
initialize_db()

# Insert sample performance
insert_performance('video1.mp4', 'Running, Passing', 'Improve passing accuracy')


3. User Interface

In [33]:
import tkinter as tk
from tkinter import filedialog, messagebox
import cv2
import mediapipe as mp
import sqlite3

# Initialize MediaPipe Pose module
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# Initialize database
def initialize_db():
    conn = sqlite3.connect('player_analysis.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS performance (
                        id INTEGER PRIMARY KEY,
                        video_path TEXT,
                        actions TEXT,
                        feedback TEXT)''')
    conn.commit()
    conn.close()

# Insert performance into the database
def insert_performance(video_path, actions, feedback):
    conn = sqlite3.connect('player_analysis.db')
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO performance (video_path, actions, feedback) 
                      VALUES (?, ?, ?)''', (video_path, actions, feedback))
    conn.commit()
    conn.close()

def pose_estimation_on_frame(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    annotated_frame = frame.copy()
    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(annotated_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    return annotated_frame, results.pose_landmarks

def recognize_action(landmarks):
    if landmarks is not None:
        left_shoulder = landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        left_elbow = landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
        left_wrist = landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]

        shoulder_angle = calculate_angle(
            (left_shoulder.x, left_shoulder.y),
            (left_elbow.x, left_elbow.y),
            (left_wrist.x, left_wrist.y)
        )

        if shoulder_angle < 45:
            return "Action: Shooting"
        elif shoulder_angle > 135:
            return "Action: Running"

    return "Action: Unknown"

def calculate_angle(a, b, c):
    import math
    ang = math.degrees(math.atan2(c[1] - b[1], c[0] - b[0]) - math.atan2(a[1] - b[1], a[0] - b[0]))
    return abs(ang) % 360

def process_video(video_file):
    cap = cv2.VideoCapture(video_file)
    actions = []
    feedback = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        annotated_frame, landmarks = pose_estimation_on_frame(frame)
        action = recognize_action(landmarks)
        actions.append(action)
        feedback.append(generate_feedback(action))

    cap.release()
    return actions, feedback

def generate_feedback(action):
    if action == "Action: Shooting":
        return "Feedback: Focus on improving your shooting technique. Aim for accuracy."
    elif action == "Action: Running":
        return "Feedback: Maintain a steady pace and good form while running."
    else:
        return "Feedback: Keep practicing your actions for better performance."

def analyze_video():
    video_path = filedialog.askopenfilename(title="Select Video File",
                                             filetypes=[("Video Files", "*.mp4;*.avi;*.mov")])
    if not video_path:
        return

    actions, feedback = process_video(video_path)
    # Log the last action and feedback
    if actions and feedback:
        insert_performance(video_path, ', '.join(actions), ', '.join(feedback))
        messagebox.showinfo("Analysis Complete", f"Actions: {', '.join(actions)}\nFeedback: {', '.join(feedback)}")
    else:
        messagebox.showinfo("Analysis Complete", "No actions detected.")

# Create the main application window
root = tk.Tk()
root.title("Video Analysis Tool")
root.geometry("400x200")

initialize_db()  # Initialize the database

# Add a button to analyze video
analyze_button = tk.Button(root, text="Analyze Video", command=analyze_video)
analyze_button.pack(pady=20)




In [34]:
# Start the Tkinter event loop
root.mainloop()